# Yanex Results API Demo

This notebook demonstrates the Yanex Results API using a physics simulation theme - projectile motion analysis.

We'll run experiments with different launch angles and velocities, then use the Results API to analyze and compare the results.

## Setup and Run Experiments

First, let's create some toy experiments simulating projectile motion with different parameters.

In [30]:
import math
import os
import time
from pathlib import Path

import yanex


def simulate_projectile(angle, velocity):
    """Simulate projectile motion and return key metrics."""
    # Convert angle to radians
    angle_rad = math.radians(angle)
    g = 9.81  # gravity

    # Calculate trajectory metrics
    time_of_flight = 2 * velocity * math.sin(angle_rad) / g
    max_range = velocity**2 * math.sin(2 * angle_rad) / g
    max_height = (velocity * math.sin(angle_rad)) ** 2 / (2 * g)

    return {
        "max_range": max_range,
        "max_height": max_height,
        "time_of_flight": time_of_flight,
    }


# Run experiments with different parameters
experiment_configs = [
    {"angle": 30, "velocity": 20},
    {"angle": 45, "velocity": 20},
    {"angle": 60, "velocity": 20},
    {"angle": 45, "velocity": 15},
    {"angle": 45, "velocity": 25},
]

experiment_ids = []

for config in experiment_configs:
    with yanex.create_experiment(
        script_path=Path(os.path.abspath("")),
        config=config,
        name=f"projectile_angle_{config['angle']}_vel_{config['velocity']}",
        tags=["projectile", "simulation", "results-api-demo"],
        allow_dirty=True,
    ) as exp:
        # Simulate the experiment with some steps
        results = simulate_projectile(config["angle"], config["velocity"])
        time.sleep(0.1)  # Simulate some work

        # Log results
        yanex.log_metrics(results)

        experiment_ids.append(exp.experiment_id)
        print(
            f"Completed experiment {exp.experiment_id}: angle={config['angle']}°, velocity={config['velocity']}m/s"
        )

print(f"\nRan {len(experiment_ids)} experiments")

Completed experiment 25fc749d: angle=30°, velocity=20m/s
✓ Experiment completed successfully: 25fc749d
  Directory: /Users/thomas/.yanex/experiments/25fc749d
Completed experiment 51d3aaba: angle=45°, velocity=20m/s
✓ Experiment completed successfully: 51d3aaba
  Directory: /Users/thomas/.yanex/experiments/51d3aaba
Completed experiment fcdc417a: angle=60°, velocity=20m/s
✓ Experiment completed successfully: fcdc417a
  Directory: /Users/thomas/.yanex/experiments/fcdc417a
Completed experiment cd31d506: angle=45°, velocity=15m/s
✓ Experiment completed successfully: cd31d506
  Directory: /Users/thomas/.yanex/experiments/cd31d506
Completed experiment 407482f1: angle=45°, velocity=25m/s
✓ Experiment completed successfully: 407482f1
  Directory: /Users/thomas/.yanex/experiments/407482f1

Ran 5 experiments


## Results API: Import and Basic Usage

Now let's import the Results API and explore our experimental data.

In [31]:
import yanex.results as yr

## Finding experiments with `get_experiments()`

The `get_experiments()` method is the most flexible way to search for experiments. It returns experiment objects that match your criteria.

In [32]:
# Find all experiments
all_experiments = yr.get_experiments()
print(f"Total experiments found: {len(all_experiments)}")

# Find experiments by name pattern
projectile_experiments = yr.get_experiments(name="*angle_45*")
print(f"\nExperiments with 45° angle: {len(projectile_experiments)}")
for exp in projectile_experiments:
    print(f" - {exp.id}: {exp.name} (ran for {exp.duration} seconds)")

Total experiments found: 85

Experiments with 45° angle: 3
 - 407482f1: projectile_angle_45_vel_25 (ran for 0:00:00.106593 seconds)
 - cd31d506: projectile_angle_45_vel_15 (ran for 0:00:00.105823 seconds)
 - 51d3aaba: projectile_angle_45_vel_20 (ran for 0:00:00.102742 seconds)


## Experiment Object

The `Experiment` object provides methods to access metrics, parameters, and other details of the experiment.

In [ ]:
exp = all_experiments[0]
print(f"\nExperiment {exp.id} details:")
print(f"Name: {exp.name}")
print(f"Description: {exp.description}")
print(f"Status: {exp.status}")
print(f"Tags: {exp.tags}")
print(f"Started At: {exp.started_at}")
print(f"Completed At: {exp.completed_at}")
print(f"Duration: {exp.duration}")
print(f"Script Path: {exp.script_path}")
print(f"Archived: {exp.archived}")
print(f"Experiment Directory: {exp.experiment_dir}")

print("\n----------\n")

print("Experiment Parameters")
for key, value in exp.get_params().items():
    print(f"  {key}: {value}")

print("\nExperiment Metrics")
print(exp.get_metrics())
print("max range", exp.get_metric("max_range"))


Experiment 407482f1 details:
Name: projectile_angle_45_vel_25
Description: None
Status: completed
Tags: ['projectile', 'results-api-demo', 'simulation']
Started At: 2025-08-19 07:55:39.136445+00:00
Completed At: 2025-08-19 07:55:39.243038+00:00
Duration: 0:00:00.106593
Script Path: /Users/thomas/code/yanex/examples
Archived: False
Experiment Directory: /Users/thomas/.yanex/experiments/407482f1

----------

Experiment Parameters
  angle: 45
  velocity: 25
Experiment Metrics
[{'max_range': 63.710499490316, 'max_height': 15.927624872578994, 'time_of_flight': 3.6040100977907614, 'step': 0, 'timestamp': '2025-08-19T07:55:39.241849'}]
max range 63.710499490316


## Using compare() Method - DataFrame Creation

The `compare()` method is powerful for analysis - it creates a pandas DataFrame with all experiment data for easy comparison and visualization.

The DataFrame contains metadata, parameters, and metrics for each experiment, allowing for in-depth analysis and visualization. Note that the DataFrame is structured with a multi-level column index, making it easy to access specific parameters and metrics.

For example, to access the metric "max_range" column of all experiments, you can use:
```python
max_range = comparison_df[("metric", "max_range")]
```


In [4]:
# Get a comparison DataFrame of all projectile experiments
comparison_df = yr.compare(name="projectile*")

print("Experiment Comparison DataFrame:")
display(comparison_df)

print(f"\nDataFrame shape: {comparison_df.shape}")
print(f"Columns: {list(comparison_df.columns)}")

Experiment Comparison DataFrame:


category                             meta                               \
name                                 name             started duration   
experiment_id                                                            
3b29d724       projectile_angle_45_vel_25 2025-08-18 09:18:07   0 days   
3dc1173c       projectile_angle_45_vel_15 2025-08-18 09:18:07   0 days   
37fe313d       projectile_angle_60_vel_20 2025-08-18 09:18:07   0 days   
d2d4e703       projectile_angle_45_vel_20 2025-08-18 09:18:07   0 days   
7116d959       projectile_angle_30_vel_20 2025-08-18 09:18:07   0 days   

category                                                           param  \
name              status                                      tags angle   
experiment_id                                                              
3b29d724       completed  projectile, simulation, results-api-demo    45   
3dc1173c       completed  projectile, simulation, results-api-demo    45   
37fe313d       completed  projectile, simulation, results-api-demo    60   
d2d4e703       completed  projectile, simulation, results-api-demo    45   
7116d959       completed  projectile, simulation, results-api-demo    30   

category                   metric                                \
name          velocity max_height max_range step time_of_flight   
experiment_id                                                     
3b29d724            25    15.9276   63.7105    0         3.6040   
3dc1173c            15     5.7339   22.9358    0         2.1624   
37fe313d            20    15.2905   35.3119    0         3.5312   
d2d4e703            20    10.1937   40.7747    0         2.8832   
7116d959            20     5.0968   35.3119    0         2.0387   

category                                   
name                            timestamp  
experiment_id                              
3b29d724       2025-08-18T09:18:07.861820  
3dc1173c       2025-08-18T09:18:07.679245  
37fe313d       2025-08-18T09:18:07.503464  
d2d4e703       2025-08-18T09:18:07.328465  
7116d959       2025-08-18T09:18:07.145164


DataFrame shape: (5, 12)
Columns: [('meta', 'name'), ('meta', 'started'), ('meta', 'duration'), ('meta', 'status'), ('meta', 'tags'), ('param', 'angle'), ('param', 'velocity'), ('metric', 'max_height'), ('metric', 'max_range'), ('metric', 'step'), ('metric', 'time_of_flight'), ('metric', 'timestamp')]


In [5]:
# Analyze the results using pandas operations
print("=== Analysis using pandas operations ===")

# Find the experiment with maximum range
max_range_idx = comparison_df[("metric", "max_range")].idxmax()
max_range_exp = comparison_df.loc[max_range_idx]
print(
    f"Best range: {max_range_exp[('metric', 'max_range')]:.1f}m at angle={max_range_exp[('param', 'angle')]}°, velocity={max_range_exp[('param', 'velocity')]}m/s"
)

# Group by angle and show average performance
print("\n=== Performance by Launch Angle ===")
angle_summary = comparison_df.groupby(("param", "angle"))[
    [("metric", "max_range"), ("metric", "max_height")]
].mean()
display(angle_summary.round(1))

# # Group by velocity and show average performance
# print("\n=== Performance by Velocity ===")
# velocity_summary = comparison_df.groupby("velocity")[["max_range", "max_height"]].mean()
# print(velocity_summary.round(1))

=== Analysis using pandas operations ===
Best range: 63.7m at angle=45°, velocity=25m/s

=== Performance by Launch Angle ===


category          metric           
name           max_range max_height
(param, angle)                     
30                  35.3        5.1
45                  42.5       10.6
60                  35.3       15.3

## Clean up

Call the next cell to delete the demo experiments.

In [29]:
exp_count = yr.delete_experiments(tags=["results-api-demo"])
print(f"Deleted {exp_count} experiments")

Deleted 10 experiments
